In [1]:
class Question:
    def __init__(self, enonce, options, bonne_reponse):
        """
        Permet d'initialiser une question (QCM)
        Comme arguments :
        enonce : str, texte de la question
        options : list de str (list[str]), options possibles
        bonne_reponsse : list[str] ou str, bonne(s) réponse(s)
        """
        self.enonce = enonce
        self.options = options
        self.bonne_reponse = bonne_reponse
        
    def poser(self):
        """
        Permet d'afficher la question à l'utilisateur, ainsi que les options.
        Récupère ensuite les réponses sous forme de str ou list[str]
        """
        #Affiche l'énoncé de la question
        print("\n" + self.enonce)

        #Affiche les options, numérotées
        for i, option in enumerate(self.options, 1):
            print(f"{i}. {option}")
        
        #Récupère la réponse de l'utilisateur, les numéros séparés par des virgules si plusieurs choix
        reponse = input("Votre réponse (chaque numéro séparé par une virgule si plusieurs choix) :")

        #Pour convertir les numéros donnés en indices (0-based) afin de récupérer les options qui correspondent (comme l'utilisateur tape un numéro, pas le texte exact de la réponse)
        indices = [int(x.strip()) - 1 for x in reponse.split(",")]

        #Récupère donc les réponses correspondantes, choisies par l'utilisateur
        reponses_utilisateur = [self.options[i] for i in indices]
        
        #Boucle : si une seule bonne réponse, return str, else return list[str] (vérification du type de l'object avec isinstance)
        if isinstance(self.bonne_reponse, str):
            return reponses_utilisateur[0] #retourne la première (donc seule) réponse choisie
        else :
            return reponses_utilisateur #retourne la liste des réponses choisies
        
    def verifier_reponse(self, reponse_utilisateur):
        """ 
        Permet de vérifier si la réponse donnée par l'utilisateur est la bonne, ou non.
        reponse_utilisateur : str ou list[str], réponse donnée par l'utilisateur
        booléen : retourne True si la réponse est bonne, False sinon
        """
        #Si la bonne réponse est sous forme de liste, il faut comparer, comme des ensembles, les réponses données (fonction : set())
        if isinstance(self.bonne_reponse, list):
            return set(reponse_utilisateur) == set(self.bonne_reponse)
        else:
            #sinon, il faut comparer les deux strings
            return reponse_utilisateur == self.bonne_reponse
    
    def score_utilisateur(self, reponse_utilisateur):
        """
        Permet de calculer le score partiel : combien de points l'utilisateur gagne, même si sa réponse est partiellement correcte.
        S'il s'agit d'un QCM simple : 1 si la réponse est la bonne, 0 sinon.
        S'il s'agit d'un QCM multiple : score est le nombre de réponses correctes choisies.
        """
        # QCM multiple :
        if isinstance(self.bonne_reponse, list):
            return len(set(reponse_utilisateur) & set(self.bonne_reponse)) # nombre de réponses correctes choisies par l'utilisateur
        else: # QCM simple
            return int(reponse_utilisateur == self.bonne_reponse)      
        
    def to_dict(self):
        """
        Permet de convertir la question en dictionnaire pour la stocker dans un fichier JSON
        retourne : dict
        """
        return {
            "enonce": self.enonce,
            "options": self.options,
            "bonne_reponse": self.bonne_reponse
        }

    @classmethod
    #la méthode appartient à la classe et non à une instance, la première variable est cls qui représente la classe, ici Question
    def from_dict(cls, data):
        """
        Permet de créer une instance Question à partir, donc, d'un dictionnaire (JSON)
        data : dict qui contient les clés "enonce", "options" et "bonne_reponse"
        retourne : Question
        """
        return cls(
            enonce = data["enonce"],
            options = data["options"],
            bonne_reponse = data["bonne_reponse"]
        )

In [3]:
from quiz_app.question import Question

class Quiz:
    """
    Permet de représenter un quiz, avec un titre et une liste de questions.
    titre : str, titre (nom) du quiz
    questions : list[Question], la liste des questions du quiz
    """
    def __init__(self, titre):
        """
        Permet d'initialiser un quiz avec un titre et une liste de questions vide.
        (titre : str, titre du quiz)
        """
        self.titre = titre
        self.questions = [] #c'est la liste qui va contenir les objets Question du quiz
    
    def ajouter_question(self,question):
        """
        Permet d'ajouter une question à cette liste de questions du quiz.
        question : Question (instance de la classe Question), c'est la question à ajouter à la liste
        """
        self.questions.append(question)

    def lancer(self):
        """
        Permet de lancer le quiz, le faire passer à l'utilisateur.
        Pour chaque question, on affiche l'énoncé et les options, on récupère la réponse donnée par l'utilisateur, on vérifie si celle-ci est correcte, et on affiche "Correct" ou "Faux" ainsi que le score actuel.
        Enfin, à la fin on affiche le score total (chque réponse correcte rapporte 1 point)
        return : int, soit le score total
        """
        print(f"\n== QUIZ : {self.titre} ==\n") # on affiche le titre
        score = 0 # on initialise le score à 0
        max_score = self.score_max() # le score max possible

        # On parcourt toutes les questions du quiz, avec un compteur (boucle for avec enumerate)
        for i, question in enumerate(self.questions, 1):
            print(f"\nQuestion {i}/{len(self.questions)}") # permet d'afficher le numéro de la question et le nombre total de questions posées

            # On pose la question à l'utilisateur avant de récupérer sa réponse (avec la méthode poser de Question)
            reponse = question.poser()

            # On calcule le score partiel
            points = question.score_utilisateur(reponse)
            score += points

            if points == 0:
                print (f"Faux... La bonne réponse est : {question.bonne_reponse}")
            elif isinstance(question.bonne_reponse, list) and points < len(question.bonne_reponse):
                print (f"Partiellement correct. Score : (+{points} point{'s' if points > 1 else ''})")
            else:
                print (f"Correct ! Score : (+{points} point{'s' if points > 1 else ''})")
            
            print ("-"*40) # permet d'afficher une ligne de tirets, pour faire plus propre

        print (f"\n Score final : {score} / {max_score()}")
        return score
    
    def score_max(self):
        """
        Permet de calculer le score max possible du quiz.
        Le QCM simple rapporte 1 point par question, celui multiple, autant que le nombre de bonnes réponses.
        """
        total = 0
        for q in self.questions:
            if isinstance((q.bonne_reponse, list)):
                total += len(q.bonne_reponse)
            else:
                total += 1
        return total
    
    def to_dict(self):
        """Pour sérialiser le quiz, afin de le sauvegarder sous forme de fichier JSON"""
        return {
            "titre" : self.titre,
            "questions" : [q.to_dict() for q in self.question]
        }
    
    @classmethod
    def from_dict(cls, data):
        """
        Permet de créer une instance Quiz à partir, donc, d'un dictionnaire (JSON)
        retourne : Quiz
        """
        quiz = cls(data["titre"]) # permet de créer une nouvelle instance de quiz, avec le titre de celui-ci stocké dans data
        for q_data in data ["questions"]:
            quiz.ajouter_question(Question.from_dict(q_data)) # pour contenir toutes les questions sous forme d'objets Question
        return quiz

In [4]:
class Player:
    """
    Permet de représenter un joueur qui passe un quiz.
    """
    def __init__(self, nom):
        """
        Permet d'initialiser le joueur, avec un nom et un dictionnaire pour ses scores.
        nom : str, le nom du joueur
        """
        self.nom = nom
        self.scores = {} # les données sont alors stockées de la façon : {titre_quiz: score_obtenu}
    
    def enregistrer_score(self, quiz, score):
        """
        Permet d'enregistrer un score d'un joueur, pour un quiz défini.
        quiz : Quiz, instance du quiz que le joueur a passé.
        score : int, score obtenu par le joueur dans pour ce quiz.
        """
        self.scores[quiz.titre] = score

    def afficher_scores(self):
        """
        Permet d'afficher la totalité des scores obtenus par le joueur.
        """
        print (f"\n Scores de {self.nom} :")
        for titre, score in self.scores.items(): # self.scores renvoie une vue de chque éléments (items), ici (titre_quiz, score_obtenu)
            print (f"{titre} : {score}")